# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

The easiest way to install the required libraries is with [Conda](https://docs.conda.io/projects/conda/en/latest/user-guide/overview.html). Create a new environment, then install the dependencies in that environment. In your terminal:

```bash
conda create -n pysyft python=3
conda activate pysyft # some older version of conda require "source activate pysyft" instead.
conda install jupyter notebook
pip install syft
pip install numpy
```

If you have any errors relating to zstd - run the following (if everything above installed fine then skip this step):

```
pip install --upgrade --force-reinstall zstd
```

and then retry installing syft (pip install syft).

If you are using Windows, I suggest installing [Anaconda and using the Anaconda Prompt](https://docs.anaconda.com/anaconda/user-guide/getting-started/) to work from the command line. 

With this environment activated and in the repo directory, launch Jupyter Notebook:

```bash
jupyter notebook
```

and re-open this notebook on the new Jupyter server.

If any part of this doesn't work for you (or any of the tests fail) - first check the [README](https://github.com/OpenMined/PySyft.git) for installation help and then open a Github Issue or ping the #beginner channel in our slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
import torch as th

In [2]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [3]:
y = x + x

In [4]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [5]:
import syft as sy

W0725 11:09:03.482582  4528 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0). Fix this by compiling custom ops.
W0725 11:09:03.928555  4528 deprecation_wrapper.py:119] From C:\Users\Desh\Anaconda3\envs\pysyft\lib\site-packages\tf_encrypted\session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [6]:

hook = sy.TorchHook(th)

In [7]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [21]:
bob = sy.VirtualWorker(hook, id="bob")

In [22]:
bob._objects

{}

In [23]:
x = th.tensor([1,2,3,4,5])

In [24]:
x = x.send(bob)

In [25]:
bob._objects

{99144252162: tensor([1, 2, 3, 4, 5])}

In [26]:
x.location

<VirtualWorker id:bob #objects:1>

In [27]:
x.id_at_location

99144252162

In [28]:
x.id

70537631961

In [29]:
x.owner

<VirtualWorker id:me #objects:0>

In [30]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [31]:
x

(Wrapper)>[PointerTensor | me:70537631961 -> bob:99144252162]

In [32]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [33]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [34]:
# Create two workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [35]:
x = th.tensor([1,2,3,4,5])

In [36]:
x = x.send(bob, alice)

In [37]:
bob._objects

{48191337660: tensor([1, 2, 3, 4, 5])}

In [38]:
alice._objects

{34836169451: tensor([1, 2, 3, 4, 5])}

In [39]:
x = x.get()

In [40]:
x

[tensor([1, 2, 3, 4, 5]), tensor([1, 2, 3, 4, 5])]

In [41]:
bob._objects

{}

In [29]:
alice._objects

{}

# Lesson: Introducing Remote Arithmetic

In [42]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [43]:
x

(Wrapper)>[PointerTensor | me:25530077040 -> bob:41240428463]

In [44]:
y

(Wrapper)>[PointerTensor | me:28601195100 -> bob:12001439767]

In [45]:
z = x + y

In [46]:
z

(Wrapper)>[PointerTensor | me:37220932493 -> bob:18255835202]

In [47]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [48]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:83789004615 -> bob:17593987698]

In [49]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [50]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [51]:
z = (x + y).sum()

In [52]:
z.backward()

(Wrapper)>[PointerTensor | me:41168504505 -> bob:3331004926]

In [53]:
x = x.get()

In [54]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [55]:
x.grad

tensor([1., 1., 1., 1., 1.])

In [56]:
z.get()

tensor(20., requires_grad=True)

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [57]:
input_tensor = th.tensor([[1., 1], [0, 1], [1, 0], [0, 0]], requires_grad=True).send(bob)
target_tensor = th.tensor([[1.], [1], [0], [0]], requires_grad=True).send(bob)

In [58]:
weights = th.tensor([[0.], [0.]], requires_grad=True).send(bob)

In [59]:
bob._objects

{41240428463: tensor([1, 2, 3, 4, 5]),
 12001439767: tensor([1, 1, 1, 1, 1]),
 63232931240: tensor([1., 1., 1., 1., 1.], requires_grad=True),
 40574896888: tensor([[1., 1.],
         [0., 1.],
         [1., 0.],
         [0., 0.]], requires_grad=True),
 98377194298: tensor([[1.],
         [1.],
         [0.],
         [0.]], requires_grad=True),
 69749393259: tensor([[0.],
         [0.]], requires_grad=True)}

In [60]:
# Train Model
for i in range(10):
    pred = input_tensor.mm(weights)
    
    loss = ((pred-target_tensor)**2).sum()
    loss.backward()
    weights.data.sub_(weights.grad*0.1)
    weights.grad*=0
    print(loss.get().data)
    

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [1]:
import torch as th
import syft as sy
hook = sy.TorchHook(th)

W0725 11:43:04.866904  9984 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0). Fix this by compiling custom ops.
W0725 11:43:04.926936  9984 deprecation_wrapper.py:119] From C:\Users\Desh\Anaconda3\envs\pysyft\lib\site-packages\tf_encrypted\session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
# Create two workers
bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")

In [3]:
bob._objects

{}

In [4]:
x = th.tensor([1, 2, 3, 4, 5]).send(bob)

In [5]:
x

(Wrapper)>[PointerTensor | me:32102953901 -> bob:9209774886]

In [6]:
bob._objects

{9209774886: tensor([1, 2, 3, 4, 5])}

In [7]:
del x

In [8]:
bob._objects

{9209774886: tensor([1, 2, 3, 4, 5])}

In [75]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [76]:
bob._objects

{}

In [73]:
x = "asdf"

In [74]:
bob._objects

{}

In [54]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [55]:
x

(Wrapper)>[PointerTensor | me:19384969793 -> bob:49166137090]

In [56]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [57]:
x = "asdf"

In [58]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [59]:
del x

In [60]:
bob._objects

{49166137090: tensor([1, 2, 3, 4, 5])}

In [61]:
bob = bob.clear_objects()
bob._objects

{}

In [62]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [63]:
bob._objects

{17426510898: tensor([1, 2, 3, 4, 5])}

In [64]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [65]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

In [130]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [66]:
z = x + y

TensorsNotCollocatedException: You tried to call a method involving two tensors where one tensor is actually locatedon another machine (is a PointerTensor). Call .get() on the PointerTensor or .send(bob) on the other tensor.

Tensor A: [PointerTensor | me:46419059800 -> bob:14412738960]
Tensor B: tensor([1, 1, 1, 1, 1])

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [3]:
from torch import nn, optim

In [4]:
nn.Linear(2, 1)

Linear(in_features=2, out_features=1, bias=True)

In [5]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

NameError: name 'th' is not defined

In [70]:
# A Toy Model
model = nn.Linear(2,1)

In [71]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [85]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(0.0860)
tensor(0.0586)
tensor(0.0402)
tensor(0.0278)
tensor(0.0194)
tensor(0.0136)
tensor(0.0096)
tensor(0.0069)
tensor(0.0049)
tensor(0.0036)
tensor(0.0026)
tensor(0.0019)
tensor(0.0014)
tensor(0.0010)
tensor(0.0008)
tensor(0.0006)
tensor(0.0004)
tensor(0.0003)
tensor(0.0002)
tensor(0.0002)


In [89]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [90]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [91]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [122]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [123]:
train()

tensor(0.0824, requires_grad=True)
tensor(0.0006, requires_grad=True)
tensor(5.0509e-06, requires_grad=True)
tensor(1.8660e-07, requires_grad=True)
tensor(1.2442e-07, requires_grad=True)
tensor(1.0333e-07, requires_grad=True)
tensor(8.5991e-08, requires_grad=True)
tensor(7.1573e-08, requires_grad=True)
tensor(5.9562e-08, requires_grad=True)
tensor(4.9569e-08, requires_grad=True)
tensor(4.1269e-08, requires_grad=True)
tensor(3.4340e-08, requires_grad=True)
tensor(2.8568e-08, requires_grad=True)
tensor(2.3789e-08, requires_grad=True)
tensor(1.9802e-08, requires_grad=True)
tensor(1.6471e-08, requires_grad=True)
tensor(1.3715e-08, requires_grad=True)
tensor(1.1405e-08, requires_grad=True)
tensor(9.4892e-09, requires_grad=True)
tensor(7.9073e-09, requires_grad=True)
tensor(6.5776e-09, requires_grad=True)
tensor(5.4857e-09, requires_grad=True)
tensor(4.5577e-09, requires_grad=True)
tensor(3.8027e-09, requires_grad=True)
tensor(3.1617e-09, requires_grad=True)
tensor(2.6215e-09, requires_grad=

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [163]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:bob #tensors:0>

<VirtualWorker id:alice #tensors:0>

In [167]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [170]:
x = x.send(alice)

In [171]:
bob._objects

{21931995238: tensor([1, 2, 3, 4, 5])}

In [172]:
alice._objects

{55299383133: (Wrapper)>[PointerTensor | alice:55299383133 -> bob:21931995238]}

In [174]:
y = x + x

In [175]:
y

(Wrapper)>[PointerTensor | me:48924169518 -> alice:48924169518]

In [176]:
bob._objects

{21931995238: tensor([1, 2, 3, 4, 5]),
 48924169518: tensor([ 2,  4,  6,  8, 10])}

In [177]:
alice._objects

{48924169518: (Wrapper)>[PointerTensor | alice:48924169518 -> bob:48924169518],
 55299383133: (Wrapper)>[PointerTensor | alice:55299383133 -> bob:21931995238]}

In [178]:
jon = sy.VirtualWorker(hook, id="jon")

In [186]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [187]:
bob._objects

{10539507281: tensor([1, 2, 3, 4, 5])}

In [188]:
alice._objects

{40541026396: (Wrapper)>[PointerTensor | alice:40541026396 -> bob:10539507281]}

In [189]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:40541026396 -> bob:10539507281]

In [190]:
bob._objects

{10539507281: tensor([1, 2, 3, 4, 5])}

In [191]:
alice._objects

{}

In [192]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [193]:
bob._objects

{}

In [194]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [195]:
bob._objects

{28843833290: tensor([1, 2, 3, 4, 5])}

In [196]:
alice._objects

{88814770977: (Wrapper)>[PointerTensor | alice:88814770977 -> bob:28843833290]}

In [197]:
del x

In [198]:
bob._objects

{}

In [199]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [212]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #tensors:0>

In [213]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [214]:
bob._objects

{81966670653: tensor([1, 2, 3, 4, 5])}

In [215]:
alice._objects

{}

In [216]:
x.move(alice)

(Wrapper)>[PointerTensor | me:547575813 -> alice:547575813]

In [217]:
bob._objects

{}

In [218]:
alice._objects

{547575813: tensor([1, 2, 3, 4, 5])}

In [202]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [203]:
bob._objects

{75487377866: tensor([1, 2, 3, 4, 5])}

In [204]:
alice._objects

{94092707138: (Wrapper)>[PointerTensor | alice:94092707138 -> bob:75487377866]}

In [205]:
x.remote_get()

(Wrapper)>[PointerTensor | me:87205391815 -> alice:94092707138]

In [206]:
bob._objects

{}

In [207]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}

In [208]:
x.move(bob)

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [209]:
x

(Wrapper)>[PointerTensor | me:87205391815 -> bob:87205391815]

In [210]:
bob._objects

{87205391815: tensor([1, 2, 3, 4, 5])}

In [211]:
alice._objects

{94092707138: tensor([1, 2, 3, 4, 5])}

## Project Implementation (Training models using federated learning)

In [1]:
# Regular Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
!pip install multiprocess
from multiprocess import Pool, TimeoutError, cpu_count

  Using cached https://files.pythonhosted.org/packages/8f/1c/60b2259979898b93693fbc7ffcbe57dc4339fc609d58e11bf19083aee155/multiprocess-0.70.8-cp37-cp37m-win_amd64.whl


In [3]:
# Import pysyft to conduct federated learning
import syft as sy
hook = sy.TorchHook(torch)

W0803 22:49:17.151122 14624 secure_random.py:22] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow (1.14.0). Fix this by compiling custom ops.
W0803 22:49:17.210126 14624 deprecation_wrapper.py:119] From C:\Users\Desh\Anaconda3\envs\pysyft\lib\site-packages\tf_encrypted\session.py:28: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [4]:
# Create new remote Workers
worker_names = ["apples", "bananas"]
for i in range(len(worker_names)):
    worker_names[i] = sy.VirtualWorker(hook, id=worker_names[i])

In [5]:
# print(**worker_names)

In [6]:
# Define transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Define the batch size for the loader
batch_size = 64
test_batch_size = 1000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define a data loader
federated_train_loader = sy.FederatedDataLoader(
    datasets.MNIST('../data', train=True, download=True,transform=transform).federate((worker_names[0], worker_names[1])),
    batch_size=batch_size, shuffle=True, num_workers=1, pin_memory=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transform),
    batch_size=test_batch_size, shuffle=True, num_workers=1, pin_memory=True)

W0803 22:49:42.022296 14624 dataloader.py:197] The following options are not supported: num_workers: 1, pin_memory: True


In [7]:
#define training model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [8]:
# Define the training function
log_interval = 10

def train(model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        if batch_idx % log_interval == 0:
            loss = loss.get() # <-- NEW: get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * batch_size, len(federated_train_loader) * batch_size, 
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [9]:
# Define function for test
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() 
            pred = output.argmax(1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:
# Train the model
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01) # TODO momentum is not supported at the moment

for epoch in range(1, 10 + 1):
    train(model, device, federated_train_loader, optimizer, epoch)
    test(model, device, test_loader)

torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.307921
Train Epoch: 1 [640/60032 (1%)]	Loss: 2.290793
Train Epoch: 1 [1280/60032 (2%)]	Loss: 2.276487
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.245527
Train Epoch: 1 [2560/60032 (4%)]	Loss: 2.233449
Train Epoch: 1 [3200/60032 (5%)]	Loss: 2.212582
Train Epoch: 1 [3840/60032 (6%)]	Loss: 2.162882
Train Epoch: 1 [4480/60032 (7%)]	Loss: 2.145984
Train Epoch: 1 [5120/60032 (9%)]	Loss: 2.066660
Train Epoch: 1 [5760/60032 (10%)]	Loss: 2.005525
Train Epoch: 1 [6400/60032 (11%)]	Loss: 1.904129
Train Epoch: 1 [7040/60032 (12%)]	Loss: 1.734882
Train Epoch: 1 [7680/60032 (13%)]	Loss: 1.613194
Train Epoch: 1 [8320/60032 (14%)]	Loss: 1.422126
Train Epoch: 1 [8960/60032 (15%)]	Loss: 1.087193
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.937543
Train Epoch: 1 [10240/60032 (17%)]	Loss: 0.904052
Train Epoch: 1 [10880/60032 (18%)]	Loss: 0.771925
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.859318
Train Epoch: 1 [12160/60032 (20%)]	Loss: 0.535357
Train Epoch: 1 [12800/60032 (

PicklingError: Can't pickle <class 'syft.frameworks.torch.tensors.interpreters.native.Tensor'>: attribute lookup Tensor on syft.frameworks.torch.tensors.interpreters.native failed